In [1]:
library(tidyverse)
library(lubridate)
library(ggplot2)
library(sf)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.2     v tibble    3.3.0
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.1.0     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'sf' was built under R version 4.5.2"
Linking to GEOS 3.13.1, GDAL 3.11.4, PROJ 9.7.0; sf_use_s2() is TRUE



In [2]:
# Load 2024 data
df_2024 <- read_csv("../data/offenses_known_csv_1960_2024_month/offenses_known_monthly_2024.csv")|>
    glimpse()

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 309168 Columns: 197
-- Column specification --------------------------------------------------------
Delimiter: ","
chr   (35): ori, ori9, agency_name, state, state_abb, month, last_month_repo...
dbl  (153): year, number_of_months_reported, month_missing, longitude, latit...
lgl    (8): number_of_months_missing, covered_by_population_group, populatio...
date   (1): date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 309,168
Columns: 197
$ ori                                     <chr> "AK00101", "AK00101", "AK00101~
$ ori9                                    <chr> "AK0010100", "AK0010100", "AK0~
$ agency_name                             <chr> "anchorage", "anchorage", "anc~
$ state                                   <chr> "alaska", "alaska", "alaska", ~
$ state_abb                               <chr> "AK", "AK", "AK", "AK", "AK", ~
$ year                                    <dbl> 2024, 2024, 2024, 2024, 2024, ~
$ month                                   <chr> "january", "february", "march"~
$ date                                    <date> 2024-01-01, 2024-02-01, 2024-~
$ number_of_months_missing                <lgl> NA, NA, NA, NA, NA, NA, NA, NA~
$ number_of_months_reported               <dbl> 12, 12, 12, 12, 12, 12, 12, 12~
$ last_month_reported                     <chr> "december", "december", "decem~
$ month_missing                           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ fips_state_

In [3]:
# Identify all unique states in the dataset
unique_states <- df_2024 |> 
    distinct(state) |> 
    pull(state)

print(length(unique_states))

[1] 57


In [4]:
# identify all unique census regions in the 2024 data
unique_regions <- df_2024 |> 
    distinct(census_name) |> 
    pull(census_name)

# Print the unique census regions
print(length(unique_regions))

[1] 13070


In [ ]:
# Trim the df
df_2024_trimmed <- df_2024 |>
  filter(!state_abb %in% c("AS", "GU", "MP", "PR", "VI")) |> # Remove territories
  mutate(geoid = str_c(fips_state_code, fips_place_code)) |>
  # Select identification columns and the value columns we want to pivot
  select(geoid, state, state_abb, address_city, census_name, year, month, date, population, population_group,
      core_city_indication, officers_killed_by_felony:officers_assaulted, 
      starts_with("actual_"),
      starts_with("total_cleared_")) |>
  # Pivot both actual and cleared counts simultaneously
  pivot_longer(
    cols = c(starts_with("actual_"), starts_with("total_cleared_")),
    names_to = c(".value", "offense_type"),
    names_pattern = "(actual|total_cleared)_(.*)"
  ) |>
  # The columns are now named 'actual' and 'total_cleared' based on the regex group 1.
  # We rename them to match your desired output variables.
  rename(actual_count = actual, cleared_count = total_cleared) |>
  mutate(
    crime_rate = if_else(population > 0, actual_count / population * 100000, NA_real_),
    clear_occurrence_ratio = if_else(!is.na(actual_count) & actual_count > 0,
                 cleared_count / actual_count,
                 NA_real_)
  ) |>
  select(-cleared_count) |>
  glimpse()


Rows: 9,571,188
Columns: 18
$ geoid                       <chr> "0203000", "0203000", "0203000", "0203000"~
$ state                       <chr> "alaska", "alaska", "alaska", "alaska", "a~
$ state_abb                   <chr> "AK", "AK", "AK", "AK", "AK", "AK", "AK", ~
$ address_city                <chr> "anchorage", "anchorage", "anchorage", "an~
$ census_name                 <chr> "anchorage municipality", "anchorage munic~
$ year                        <dbl> 2024, 2024, 2024, 2024, 2024, 2024, 2024, ~
$ month                       <chr> "january", "january", "january", "january"~
$ date                        <date> 2024-01-01, 2024-01-01, 2024-01-01, 2024-~
$ population                  <dbl> 286958, 286958, 286958, 286958, 286958, 28~
$ population_group            <chr> "city 250,000 thru 499,999", "city 250,000~
$ core_city_indication        <chr> "core city of msa", "core city of msa", "c~
$ officers_killed_by_felony   <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ officers_k

In [15]:
# Check if there are clearance rates greater than 1
clearance_issues <- df_2024_trimmed |>
  filter(!is.na(clearance_rate) & clearance_rate > 1)
print(nrow(clearance_issues))

[1] 29654


In [16]:
# Investigate obs where clearance rate > 1
if (nrow(clearance_issues) > 0) {
  print(clearance_issues)
}

# A tibble: 29,654 x 18
   geoid   state  state_abb address_city census_name       year month date      
   <chr>   <chr>  <chr>     <chr>        <chr>            <dbl> <chr> <date>    
 1 0203000 alaska AK        anchorage    anchorage munic~  2024 febr~ 2024-02-01
 2 0203000 alaska AK        anchorage    anchorage munic~  2024 sept~ 2024-09-01
 3 0203000 alaska AK        anchorage    anchorage munic~  2024 nove~ 2024-11-01
 4 0224230 alaska AK        fairbanks    fairbanks city    2024 dece~ 2024-12-01
 5 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
 6 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
 7 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
 8 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
 9 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
10 0240950 alaska AK        kodiak       kodiak city       2024 june  2024-06-01
# i 

In [6]:
# Investigate NA in the trimmed df
na_counts <- sapply(df_2024_trimmed, function(x) sum(is.na(x) | (x == 0)))|>
    as.data.frame()

print(na_counts)

                            sapply(df_2024_trimmed, function(x) sum(is.na(x) | (x == 0)))
geoid                                                                             1026816
state                                                                              108864
state_abb                                                                          110544
address_city                                                                      1248240
census_name                                                                       1248240
year                                                                                    0
month                                                                                   0
date                                                                                    0
population                                                                        2944032
core_city_indication                                                                    0
officers_k

In [7]:
# Summarize crime count by type
crime_type_summary <- df_2024_trimmed|>
    group_by(offense_type)|>
    summarize(count = sum(offense_count),
            mean = mean(offense_count),
            sd = sd(offense_count))|>
    arrange(by = count)

crime_type_summary

offense_type,count,mean,sd
<chr>,<dbl>,<dbl>,<dbl>
actual_theft_under50_dollar,0,0.000000000,0.0000000
actual_manslaughter,2015,0.006526358,0.1074079
actual_rape_attempted,4366,0.014140982,0.1994192
actual_murder,16013,0.051864304,0.6280145
actual_robbery_with_a_knife,17433,0.056463524,1.6144173
actual_robbery_other_weapon,23976,0.077655564,1.4466047
actual_burglary_attempted,49926,0.161704691,2.0313742
actual_motor_vehicle_theft_other,61949,0.200645834,2.0720278
actual_robbery_with_a_gun,64096,0.207599725,3.4192599


In [8]:
# Get the stat for chicago
chicago_2024 <- df_2024_trimmed|>
    filter(address_city == "chicago")|>
    group_by(census_name, offense_type)|>
    summarize(total_crime = sum(offense_count))|>
    filter(offense_type %in% c("actual_all_crimes", "actual_murder", "actual_assault_total", "actual_theft_total", "actual_robbery_total"))

chicago_2024

`summarise()` has grouped output by 'census_name'. You can override using the
`.groups` argument.


census_name,offense_type,total_crime
<chr>,<chr>,<dbl>
chicago city,actual_all_crimes,184257
chicago city,actual_assault_total,81772
chicago city,actual_murder,461
chicago city,actual_robbery_total,8847
chicago city,actual_theft_total,61185
chicago transit authority,actual_all_crimes,0
chicago transit authority,actual_assault_total,0
chicago transit authority,actual_murder,0
chicago transit authority,actual_robbery_total,0


In [9]:
# Read in the us place names from a txt file
place_names <- read_csv("../data/us_place_names.csv")|>
    glimpse()

Rows: 32041 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): place_id, place_names

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 32,041
Columns: 2
$ place_id    <chr> "3583340", "3533290", "3512150", "3562620", "3570270", "35~
$ place_names <chr> "wagon mound", "hope", "carlsbad", "reserve", "santa clara~


In [10]:
# Verify if the city names match
census_names_2024 <- df_2024|>
    mutate(geoid = str_c(fips_state_code, fips_place_code))|>
    distinct(geoid)|>
    pull(geoid)

census_names_2024


[1] "0203000" "0224230" "0236400" "0238970" "0240950" "0254920" "0260310"
    [8] "0268560" "0270540" "0299230" "0286380" "0282200" "0206520" "0217410"
   [15] "0241830" "0258660" "0299185" "0268340" "0271640" "0299100" "0233140"
   [22] "0238420" "0248870" "0299090" "0299020" "0284510" "0218950" "0233360"
   [29] "0255910" "0280770" "0240400" "0217740" "0299060" "0236770" "0253050"
   [36] "0266470" "0267020" "0277690" "0283080" "0203880" "0222910" "0233800"
   [43] NA        "0199073" "0105980" "0107000" "0151696" "0125120" "0129056"
   [50] "0141968" "0148376" "0174976" "0178552" "0135800" "0128552" "0135896"
   [57] "0137864" "0161008" "0166336" "0131384" "0110065" "0109400" "0136448"
   [64] "0143336" "0176944" "0100460" "0179944" "0176680" "0151456" "0109736"
   [71] "0112040" "0139856" "0117968" "0199097" "0150000" "0162496" "0114392"
   [78] "0115064" "0168160" "0104684" "0168352" "0152608" "0182752" "0118304"
   [85] "0119744" "0199101" "0151000" "0199001" "0162328" "0103220" "0199003"
   [92] "0104660" "0125240" "0126992" "0132272" "0165208" "0144608" "0170536"
   [99] "0119648" "0173872" "0123320" "0157144" "0171976" "0199005" "0124568"
  [106] "0115376" "0115640" "0144344" "0103724" "0199007" "0113672" "0109136"
  [113] "0180928" "0183640" "0199009" "0157000" "0107456" "0115472" "0171280"
  [120] "0134480" "0199011" "0177880" "0199013" "0131912" "0129560" "0145496"
  [127] "0199015" "0101852" "0138272" "0159640" "0180352" "0157576" "0135152"
  [134] "0156472" "0199017" "0140672" "0141296" "0178204" "0199019" "0113648"
  [141] "0112760" "0142016" "0199021" "0115136" "0146504" "0138608" "0176080"
  [148] "0199023" "0111032" "0129704" "0158968" "0170416" "0176632" "0199025"
  [155] "0138152" "0175960" "0132080" "0128504" "0116264" "0199027" "0102860"
  [162] "0143264" "0199029" "0133976" "0163408" "0199031" "0123296" "0124184"
  [169] "0153856" "0140096" "0199033" "0169648" "0177280" "0153016" "0114152"
  [176] "0142160" "0143648" "0199035" "0124808" "0112496" "0164368" "0199037"
  [183] "0130640" "0165472" "0199039" "0101708" "0157120" "0126848" "0128936"
  [190] "0163768" "0143816" "0164776" "0199041" "0144728" "0109016" "0121448"
  [197] "0130160" "0199043" "0118976" "0132968" "0129032" "0130496" "0199045"
  [204] "0157648" "0119360" "0142472" "0153256" "0154480" "0148400" "0159832"
  [211] "0102428" "0115304" "0199047" "0169120" "0199049" "0127616" "0116600"
  [218] "0129608" "0134096" "0162075" "0148064" "0163336" "0132896" "0118856"
  [225] "0128672" "0174400" "0178240" "0137312" "0199051" "0181720" "0174688"
  [232] "0122816" "0117176" "0148712" "0199053" "0103004" "0109208" "0122216"
  [239] "0126824" "0199055" "0103028" "0128696" "0129992" "0163288" "0135392"
  [246] "0171832" "0168280" "0101660" "0179728" "0199057" "0125840" "0105932"
  [253] "0199059" "0167056" "0159496" "0163576" "0178984" "0135200" "0199061"
  [260] "0129464" "0167800" "0133424" "0171040" "0116240" "0199063" "0124664"
  [267] "0127376" "0199065" "0131720" "0151648" "0199067" "0100124" "0133856"
  [274] "0154600" "0199069" "0121184" "0102836" "0130760" "0116744" "0117824"
  [281] "0140072" "0175096" "0180376" "0199071" "0168736" "0109328" "0173080"
  [288] "0135632" "0160720" "0121952" "0169000" "0183664" "0170896" "0199075"
  [295] "0173728" "0178480" "0139520" "0148808" "0199077" "0126896" "0180064"
  [302] "0101756" "0167536" "0142640" "0166216" "0139784" "0199079" "0151600"
  [309] "0176584" "0118040" "0134816" "0155044" "0199081" "0103076" "0157048"
  [316] "0199083" "0102956" "0102260" "0123488" "0199085" "0127520" "0133712"
  [323] "0151520" "0181912" "0199087" "0177304" "0155440" "0128024" "0170128"
  [330] "0199089" "0137000" "0145784" "0132440" "0154168" "0157504" "0176824"
  [337] "0154864" "0199091" "0120296" "0143240" "0175936" "0125816" "0153112"
  [344] "0174304" "0199093" "0132848" "0182992" "0132224" "0132560" "0109424"
  [351] "0181480" "0104852" "0199095" "0100988" "0132416" "0102116" "0107912"
  [358] "0131096" "0121328

In [12]:
missing_id <- setdiff(census_names_2024, place_names$place_id)

length(missing_id)

[1] 5311